In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.cross_validation import train_test_split


#mnist = input_data.read_data_sets("/tmp/data/")
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

/home/chris/Documents/ml/env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
tf.reset_default_graph()

n_input = 784
n_hidden1 = 500
n_hidden2 = 100
n_hidden3 = 50
n_output = 10

X = tf.placeholder(tf.float32, shape=(None, n_input))
y = tf.placeholder(tf.int64, shape=(None), name="y")


with tf.contrib.framework.arg_scope([fully_connected], activation_fn = tf.nn.relu):
    hidden1 = fully_connected(X,n_hidden1)
    hidden2 = fully_connected(hidden1,n_hidden2)
    hidden3 = fully_connected(hidden2,n_hidden3)
    logits = fully_connected(hidden3, n_output, activation_fn=None)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(mnist.train.images, mnist.train.labels, 
                                                              test_size=0.9, random_state=42,
                                                              stratify=mnist.train.labels)

In [8]:
def next_batch(X,y,batch_size):
    n,m = X.shape
    shuffle_index = np.random.permutation(n)
    X_batch = X[shuffle_index[0:batch_size]]
    y_batch = y[shuffle_index[0:batch_size]]
    return X_batch, y_batch

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#learning_rate = 1

n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch, y_batch = next_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    #save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.8466667 Test accuracy: 0.8851
1 Train accuracy: 0.9266667 Test accuracy: 0.915
2 Train accuracy: 0.96 Test accuracy: 0.9306
3 Train accuracy: 0.9533333 Test accuracy: 0.9334
4 Train accuracy: 0.97333336 Test accuracy: 0.9329
5 Train accuracy: 0.98 Test accuracy: 0.9459
6 Train accuracy: 0.99333334 Test accuracy: 0.9466
7 Train accuracy: 1.0 Test accuracy: 0.943
8 Train accuracy: 0.99333334 Test accuracy: 0.9448
9 Train accuracy: 1.0 Test accuracy: 0.9386
